# Part 1: Neo4j & Cypher 기초 실습

**소요 시간**: 약 2시간  
**난이도**: 입문  
**목표**: Neo4j에 연결하고, Cypher 문법을 익히고, 첫 제조 Knowledge Graph를 직접 만들어봅니다.

---

## 학습 순서

1. 환경 설정 및 Neo4j 연결
2. Cypher 기초 문법 (CREATE, MATCH, WHERE, SET, DELETE)
3. 첫 제조 그래프 만들기 (7개 노드, 5개 관계)
4. 3-hop 쿼리 실습
5. 연습 문제
6. 정리 + 다음 Part 미리보기

---
## 1. 환경 설정

### 1.1 필수 패키지 설치 확인

Neo4j Python 드라이버와 환경 변수 관리를 위한 `python-dotenv`가 필요합니다.  
아래 셀을 실행하여 설치 여부를 확인하세요.

In [ ]:
# 필수 패키지 설치 확인 및 설치
import subprocess
import sys

def check_and_install(package_name, import_name=None):
    """패키지가 설치되어 있는지 확인하고, 없으면 설치합니다."""
    import_name = import_name or package_name
    try:
        __import__(import_name)
        print(f"[OK] {package_name} 설치 확인 완료")
    except ImportError:
        print(f"[설치 중] {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name, "-q"])
        print(f"[OK] {package_name} 설치 완료")

check_and_install("neo4j")
check_and_install("python-dotenv", "dotenv")

### 1.2 환경 변수 로드

프로젝트 루트에 `.env` 파일을 만들고 Neo4j 접속 정보를 넣어주세요.

```
NEO4J_URI=bolt://localhost:7687
NEO4J_USER=neo4j
NEO4J_PASSWORD=your_password_here
```

> **참고**: Neo4j Desktop 또는 AuraDB Free Tier를 사용할 수 있습니다.  
> AuraDB의 경우 URI가 `neo4j+s://xxxxx.databases.neo4j.io` 형식입니다.

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드 (상위 디렉토리에서도 탐색)
load_dotenv()
load_dotenv(dotenv_path="../.env")

NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "")

print(f"Neo4j URI: {NEO4J_URI}")
print(f"Neo4j User: {NEO4J_USER}")
print(f"Password 설정: {'OK' if NEO4J_PASSWORD else 'NOT SET - .env 파일을 확인하세요'}")

### 1.3 Neo4j 연결 테스트

드라이버를 생성하고 연결이 정상인지 확인합니다.

In [ ]:
from neo4j import GraphDatabase

# Neo4j 드라이버 생성
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# 연결 테스트
try:
    driver.verify_connectivity()
    print("[OK] Neo4j 연결 성공!")
    
    # 서버 정보 확인
    with driver.session() as session:
        result = session.run("RETURN 'Hello, Neo4j!' AS message")
        record = result.single()
        print(f"서버 응답: {record['message']}")
except Exception as e:
    print(f"[ERROR] 연결 실패: {e}")
    print("Neo4j가 실행 중인지, .env 설정이 올바른지 확인하세요.")

In [ ]:
# 편의 함수: Cypher 쿼리 실행 및 결과 출력
def run_query(query, parameters=None, print_result=True):
    """Cypher 쿼리를 실행하고 결과를 반환합니다."""
    with driver.session() as session:
        result = session.run(query, parameters or {})
        records = [record.data() for record in result]
        if print_result:
            if records:
                for i, rec in enumerate(records, 1):
                    print(f"  [{i}] {rec}")
            else:
                print("  (결과 없음)")
        return records

print("[OK] run_query 함수 준비 완료")

---
## 2. Cypher 기초 문법

Cypher는 Neo4j의 쿼리 언어입니다. SQL과 비슷하지만, **그래프 패턴**을 직관적으로 표현할 수 있습니다.

| SQL | Cypher | 설명 |
|-----|--------|------|
| `INSERT INTO` | `CREATE` | 데이터 생성 |
| `SELECT` | `MATCH ... RETURN` | 데이터 조회 |
| `WHERE` | `WHERE` | 조건 필터링 |
| `UPDATE` | `SET` | 데이터 수정 |
| `DELETE` | `DELETE` / `DETACH DELETE` | 데이터 삭제 |

### 2.1 CREATE - 노드 생성

노드는 `()` 괄호로 표현합니다. `:Label`은 노드의 타입(레이블)이고, `{속성}` 안에 데이터를 넣습니다.

```cypher
CREATE (변수명:레이블 {속성키: 값, ...})
```

In [ ]:
# 기존 실습 데이터 정리 (깨끗한 상태에서 시작)
run_query("MATCH (n) DETACH DELETE n", print_result=False)
print("[OK] 기존 데이터 모두 삭제 완료")

In [ ]:
# CREATE 예제 1: 단일 노드 생성
print("=== 노드 생성 ===")

# Person 레이블을 가진 노드 생성
run_query("""
    CREATE (p:Person {name: '김철수', age: 35, role: '엔지니어'})
    RETURN p.name AS 이름, p.age AS 나이, p.role AS 역할
""")

print("\n노드가 생성되었습니다!")

In [ ]:
# CREATE 예제 2: 여러 노드 한번에 생성
print("=== 여러 노드 생성 ===")

run_query("""
    CREATE (a:Person {name: '이영희', role: '품질관리'}),
           (b:Person {name: '박민수', role: '공정설계'})
    RETURN a.name AS 이름1, b.name AS 이름2
""")

### 2.2 CREATE - 관계 생성

관계는 `-[변수:타입]->`로 표현합니다. 화살표 방향이 관계의 방향입니다.

```cypher
(a)-[:관계타입 {속성}]->(b)
```

> **핵심**: 관계에는 반드시 **방향**이 있어야 합니다. `-->`는 되지만 `--`는 생성할 수 없습니다.

In [ ]:
# 관계 생성: 노드와 함께 생성
print("=== 노드 + 관계 동시 생성 ===")

run_query("""
    CREATE (d:Department {name: '제조1팀'})
    WITH d
    MATCH (p:Person {name: '김철수'})
    CREATE (p)-[:BELONGS_TO {since: 2020}]->(d)
    RETURN p.name AS 직원, d.name AS 부서
""")

### 2.3 MATCH - 패턴 매칭으로 데이터 조회

`MATCH`는 그래프에서 패턴을 찾는 명령입니다. SQL의 `SELECT`와 비슷하지만, **그래프 패턴**을 지정할 수 있습니다.

```cypher
MATCH (변수:레이블)
RETURN 변수.속성
```

In [ ]:
# MATCH 예제 1: 모든 Person 노드 조회
print("=== 모든 Person 조회 ===")
run_query("""
    MATCH (p:Person)
    RETURN p.name AS 이름, p.role AS 역할
""")

In [ ]:
# MATCH 예제 2: 관계를 포함한 패턴 매칭
print("=== 관계 패턴 매칭 ===")
run_query("""
    MATCH (p:Person)-[r:BELONGS_TO]->(d:Department)
    RETURN p.name AS 직원, type(r) AS 관계, d.name AS 부서, r.since AS 입사년도
""")

### 2.4 WHERE - 조건 필터링

`WHERE`는 조건을 걸어 결과를 필터링합니다. SQL과 거의 동일합니다.

In [ ]:
# WHERE 예제: 조건부 조회
print("=== WHERE 조건 필터링 ===")

# 나이가 30 이상인 사람
print("\n[1] 나이 30 이상:")
run_query("""
    MATCH (p:Person)
    WHERE p.age >= 30
    RETURN p.name AS 이름, p.age AS 나이
""")

# CONTAINS로 문자열 검색
print("\n[2] 역할에 '엔지니어' 포함:")
run_query("""
    MATCH (p:Person)
    WHERE p.role CONTAINS '엔지니어'
    RETURN p.name AS 이름, p.role AS 역할
""")

### 2.5 SET - 데이터 수정

`SET`은 기존 노드의 속성을 추가하거나 변경합니다.

In [ ]:
# SET 예제: 속성 수정
print("=== SET으로 속성 수정 ===")

# 김철수의 역할 변경
run_query("""
    MATCH (p:Person {name: '김철수'})
    SET p.role = '시니어 엔지니어', p.updated = date()
    RETURN p.name AS 이름, p.role AS 역할, p.updated AS 수정일
""")

### 2.6 DELETE - 데이터 삭제

- `DELETE`: 관계가 없는 노드만 삭제 가능
- `DETACH DELETE`: 노드 + 연결된 모든 관계를 함께 삭제

> **주의**: `DETACH DELETE`는 강력한 명령입니다. 프로덕션에서는 신중하게 사용하세요.

In [ ]:
# DELETE 예제
print("=== DELETE 예제 ===")

# 관계가 없는 노드 삭제
print("\n[1] 박민수 노드 삭제:")
run_query("""
    MATCH (p:Person {name: '박민수'})
    DELETE p
    RETURN '삭제 완료' AS 결과
""")

# 삭제 후 확인
print("\n[2] 남은 Person 확인:")
run_query("""
    MATCH (p:Person)
    RETURN p.name AS 이름
""")

---
## 3. 첫 제조 그래프 만들기

이제 Cypher 기초를 배웠으니, 실제 **제조 도메인 Knowledge Graph**를 만들어봅시다!

### 시나리오
자동차 부품(브레이크패드)을 제조하는 공장의 지식 그래프입니다.

### 그래프 구조 (7개 노드, 5개 관계 타입)

```
A공급사 --[SUPPLIES]--> 원재료 --[REQUIRES]-- 성형공정
                                                  |
프레스 --[PRODUCES]--> 브레이크패드           [USES]
                                                  |
                                              프레스
박리불량 --[CAUSED_BY]--> 프레스
마모 --[DETECTED_AT]--> 프레스
```

| 노드 | 레이블 | 설명 |
|------|--------|------|
| 브레이크패드 | Product | 최종 제품 |
| 프레스 | Equipment | 생산 장비 |
| 성형공정 | Process | 제조 공정 |
| 원재료 | Material | 원자재 |
| A공급사 | Supplier | 공급업체 |
| 박리불량 | Defect | 불량 유형 |
| 마모 | Symptom | 장비 증상 |

In [ ]:
# 기존 데이터 정리
run_query("MATCH (n) DETACH DELETE n", print_result=False)
print("[OK] 데이터베이스 초기화 완료")
print("\n이제 제조 그래프를 만듭니다...")

In [ ]:
# === 7개 노드 생성 ===
print("=== 7개 노드 생성 ===")

run_query("""
    // 제품
    CREATE (prod:Product {name: '브레이크패드', code: 'BP-001', spec: 'KS R 4021'})
    
    // 장비
    CREATE (equip:Equipment {name: '프레스', code: 'EQ-PRESS-01', capacity: 500, unit: 'ton'})
    
    // 공정
    CREATE (proc:Process {name: '성형공정', code: 'PROC-FORMING', duration_min: 45, temperature: 180})
    
    // 원재료
    CREATE (mat:Material {name: '원재료', code: 'MAT-001', type: '마찰재 혼합물'})
    
    // 공급사
    CREATE (sup:Supplier {name: 'A공급사', code: 'SUP-A', location: '경기도 화성시', grade: 'A'})
    
    // 불량
    CREATE (def:Defect {name: '박리불량', code: 'DEF-PEEL', severity: 'critical', description: '마찰재와 백플레이트 분리'})
    
    // 증상
    CREATE (sym:Symptom {name: '마모', code: 'SYM-WEAR', type: '기계적', indicator: '진동 증가'})
    
    RETURN
        prod.name AS 제품,
        equip.name AS 장비,
        proc.name AS 공정,
        mat.name AS 재료,
        sup.name AS 공급사,
        def.name AS 불량,
        sym.name AS 증상
""")

print("\n7개 노드 생성 완료!")

In [ ]:
# === 5가지 관계 생성 ===
print("=== 5가지 관계 생성 ===")

# 1. PRODUCES: 프레스 --[PRODUCES]--> 브레이크패드
print("\n[1] PRODUCES: 프레스 -> 브레이크패드")
run_query("""
    MATCH (e:Equipment {name: '프레스'}), (p:Product {name: '브레이크패드'})
    CREATE (e)-[:PRODUCES {daily_output: 1200}]->(p)
    RETURN e.name + ' --[PRODUCES]--> ' + p.name AS 관계
""")

# 2. USES: 성형공정 --[USES]--> 프레스
print("\n[2] USES: 성형공정 -> 프레스")
run_query("""
    MATCH (proc:Process {name: '성형공정'}), (e:Equipment {name: '프레스'})
    CREATE (proc)-[:USES {role: '주요장비'}]->(e)
    RETURN proc.name + ' --[USES]--> ' + e.name AS 관계
""")

# 3. REQUIRES: 성형공정 --[REQUIRES]--> 원재료
print("\n[3] REQUIRES: 성형공정 -> 원재료")
run_query("""
    MATCH (proc:Process {name: '성형공정'}), (m:Material {name: '원재료'})
    CREATE (proc)-[:REQUIRES {quantity_kg: 2.5}]->(m)
    RETURN proc.name + ' --[REQUIRES]--> ' + m.name AS 관계
""")

# 4. CAUSED_BY: 박리불량 --[CAUSED_BY]--> 프레스
print("\n[4] CAUSED_BY: 박리불량 -> 프레스")
run_query("""
    MATCH (d:Defect {name: '박리불량'}), (e:Equipment {name: '프레스'})
    CREATE (d)-[:CAUSED_BY {confidence: 0.85, reason: '압력 불균일'}]->(e)
    RETURN d.name + ' --[CAUSED_BY]--> ' + e.name AS 관계
""")

# 5. DETECTED_AT: 마모 --[DETECTED_AT]--> 프레스
print("\n[5] DETECTED_AT: 마모 -> 프레스")
run_query("""
    MATCH (s:Symptom {name: '마모'}), (e:Equipment {name: '프레스'})
    CREATE (s)-[:DETECTED_AT {sensor: '진동센서', threshold: 4.5}]->(e)
    RETURN s.name + ' --[DETECTED_AT]--> ' + e.name AS 관계
""")

print("\n=== 5가지 관계 생성 완료! ===")

In [ ]:
# 추가 관계: SUPPLIES (A공급사 -> 원재료)
print("=== 추가 관계: SUPPLIES ===")

run_query("""
    MATCH (s:Supplier {name: 'A공급사'}), (m:Material {name: '원재료'})
    CREATE (s)-[:SUPPLIES {contract_year: 2024, lead_time_days: 14}]->(m)
    RETURN s.name + ' --[SUPPLIES]--> ' + m.name AS 관계
""")

In [ ]:
# === 전체 그래프 시각화 쿼리 ===
print("=== 전체 그래프 확인 ===")
print("\n[1] 전체 노드 수:")
run_query("MATCH (n) RETURN count(n) AS 전체_노드_수")

print("\n[2] 전체 관계 수:")
run_query("MATCH ()-[r]->() RETURN count(r) AS 전체_관계_수")

print("\n[3] 레이블별 노드 수:")
run_query("""
    MATCH (n)
    RETURN labels(n)[0] AS 레이블, count(n) AS 노드수
    ORDER BY 노드수 DESC
""")

print("\n[4] 모든 관계 목록:")
run_query("""
    MATCH (a)-[r]->(b)
    RETURN labels(a)[0] + ':' + a.name AS 출발,
           type(r) AS 관계,
           labels(b)[0] + ':' + b.name AS 도착
""")

print("\n** Neo4j Browser에서 'MATCH (n)-[r]->(m) RETURN n,r,m' 실행하면 시각적으로 볼 수 있습니다! **")

---
## 4. 3-hop 쿼리 실습

Knowledge Graph의 진정한 힘은 **Multi-hop 쿼리**에 있습니다.  
여러 단계의 관계를 따라가며 숨겨진 연결을 찾아내는 것이죠.

### hop이란?
- **1-hop**: 직접 연결된 노드 (A -> B)
- **2-hop**: 한 단계 건너 연결 (A -> B -> C)
- **3-hop**: 두 단계 건너 연결 (A -> B -> C -> D)

> **핵심 인사이트**: 벡터 검색(Vector Search)은 1-hop 질문에는 강하지만,  
> 2-hop 이상부터는 정확도가 급격히 떨어집니다.  
> 이것이 바로 GraphRAG가 필요한 이유입니다!

In [ ]:
# === 1-hop 쿼리 ===
# 질문: "브레이크패드를 만드는 장비는?"
print("=" * 60)
print("1-hop 쿼리: 브레이크패드를 만드는 장비는?")
print("=" * 60)
print("\nCypher: MATCH (e)-[:PRODUCES]->(p:Product {name: '브레이크패드'})")
print("        RETURN e.name\n")

run_query("""
    MATCH (e:Equipment)-[:PRODUCES]->(p:Product {name: '브레이크패드'})
    RETURN e.name AS 장비, p.name AS 제품
""")

print("\n--> 1-hop은 벡터 검색으로도 충분히 답할 수 있습니다.")
print("    '브레이크패드 장비'로 검색하면 관련 문서를 찾을 수 있으니까요.")

In [ ]:
# === 2-hop 쿼리 ===
# 질문: "성형공정에서 사용하는 원재료의 공급사는?"
print("=" * 60)
print("2-hop 쿼리: 성형공정에서 사용하는 원재료의 공급사는?")
print("=" * 60)
print("\nCypher: MATCH (proc)-[:REQUIRES]->(m)<-[:SUPPLIES]-(s)")
print("        경로: 성형공정 -> 원재료 -> A공급사\n")

run_query("""
    MATCH (proc:Process {name: '성형공정'})-[:REQUIRES]->(m:Material)<-[:SUPPLIES]-(s:Supplier)
    RETURN proc.name AS 공정, m.name AS 재료, s.name AS 공급사
""")

print("\n--> 2-hop부터 벡터 검색이 어려워집니다.")
print("    '성형공정 원재료 공급사'를 한 청크에서 찾기 어렵기 때문입니다.")

In [ ]:
# === 3-hop 쿼리 ===
# 질문: "박리불량의 근본원인 장비가 관여하는 모든 공정은?"
print("=" * 60)
print("3-hop 쿼리: 박리불량의 근본원인 장비가 관여하는 모든 공정은?")
print("=" * 60)
print("\nCypher: MATCH (d:Defect)-[:CAUSED_BY]->(e)<-[:USES]-(proc)")
print("        경로: 박리불량 -> 프레스 -> 성형공정\n")

run_query("""
    MATCH (d:Defect {name: '박리불량'})-[:CAUSED_BY]->(e:Equipment)<-[:USES]-(proc:Process)
    RETURN d.name AS 불량, e.name AS 원인장비, proc.name AS 관련공정
""")

print("\n--> 더 복잡한 3-hop: 불량 -> 장비 -> 공정 -> 필요한 재료")
run_query("""
    MATCH (d:Defect {name: '박리불량'})
          -[:CAUSED_BY]->(e:Equipment)
          <-[:USES]-(proc:Process)
          -[:REQUIRES]->(m:Material)
    RETURN d.name AS 불량,
           e.name AS 원인장비,
           proc.name AS 관련공정,
           m.name AS 필요재료
""")

### Vector Search로 3-hop이 왜 어려운가?

위 3-hop 질문을 벡터 RAG로 풀어보겠습니다.

| 단계 | 정보 | 위치 |
|------|------|------|
| 1단계 | "박리불량은 프레스 압력 불균일로 발생" | 품질보고서_chunk_47 |
| 2단계 | "프레스는 성형공정의 주요 장비" | 공정매뉴얼_chunk_12 |
| 3단계 | "성형공정은 마찰재 혼합물 2.5kg 필요" | 자재관리_chunk_88 |

**문제점:**
- 3개의 서로 다른 문서, 서로 다른 청크에 정보가 흩어져 있음
- 벡터 검색은 "박리불량 공정 재료"로 검색해도 3개 청크를 연결하지 못함
- 설령 3개 청크가 모두 검색되더라도, LLM이 올바른 순서로 연결해야 함

**GraphRAG 해결:**
```cypher
MATCH (d:Defect)-[:CAUSED_BY]->(e)-[:USES]-(proc)-[:REQUIRES]->(m)
RETURN d, e, proc, m
```
한 줄의 Cypher로 정확한 답이 나옵니다!

In [ ]:
# === 보너스: 가변 경로 쿼리 ===
# N-hop까지의 모든 연결을 한번에 탐색
print("=" * 60)
print("보너스: 가변 경로 쿼리 (1~3 hop)")
print("=" * 60)
print("\n'프레스'에서 1~3 hop 이내의 모든 연결된 노드:\n")

run_query("""
    MATCH path = (start:Equipment {name: '프레스'})-[*1..3]-(connected)
    RETURN DISTINCT
        connected.name AS 연결된_노드,
        labels(connected)[0] AS 레이블,
        length(path) AS hop_수
    ORDER BY hop_수
""")

print("\n--> [*1..3]은 1~3 hop 범위의 가변 경로를 의미합니다.")
print("    이런 쿼리는 벡터 검색으로는 불가능합니다!")

---
## 5. 연습 문제

직접 해보세요! 아래 빈 코드 셀을 채워서 실행해보세요.

### 문제 1: 노드 3개 추가하기

다음 3개 노드를 추가하세요:
- `(열처리공정:Process {name: '열처리공정', temperature: 350, duration_min: 60})`
- `(B공급사:Supplier {name: 'B공급사', location: '충남 천안시', grade: 'B'})`
- `(크랙:Defect {name: '크랙', severity: 'major', description: '표면 균열'})`

그리고 다음 관계도 만드세요:
- 열처리공정 -[USES]-> 프레스
- B공급사 -[SUPPLIES]-> 원재료
- 크랙 -[CAUSED_BY]-> 프레스

> **힌트**: `CREATE`로 노드를 만들고, `MATCH`로 기존 노드를 찾은 뒤 `CREATE`로 관계를 연결하세요.

In [ ]:
# [연습 1] 여기에 코드를 작성하세요
# 힌트: CREATE 문으로 3개 노드를 먼저 만들고,
#        MATCH + CREATE 로 관계를 추가하세요.

# 노드 생성
# run_query("""
#     CREATE ...
# """)

# 관계 생성
# run_query("""
#     MATCH ...
#     CREATE ...
# """)

# 확인
# run_query("MATCH (n) RETURN labels(n)[0] AS 레이블, count(n) AS 수")

### 문제 2: 자신만의 쿼리 작성하기

다음 질문에 답하는 Cypher 쿼리를 작성하세요:

1. **"프레스 장비에서 발생하는 모든 불량과 증상은?"** (1-hop)
2. **"B공급사가 공급하는 재료를 사용하는 모든 공정은?"** (2-hop)
3. **"크랙 불량의 원인 장비를 사용하는 공정에서 필요한 재료의 공급사는?"** (3-hop)

> **힌트**:
> - 1번: `MATCH (x)-[:CAUSED_BY|DETECTED_AT]->(e:Equipment)` 처럼 `|`로 여러 관계 타입을 OR 조건으로 매칭할 수 있습니다.
> - 2번: 방향에 주의하세요. `<-[:SUPPLIES]-` 처럼 역방향 매칭도 가능합니다.
> - 3번: 4개 노드를 연결하는 긴 패턴을 작성하면 됩니다.

In [ ]:
# [연습 2-1] 프레스 장비에서 발생하는 모든 불량과 증상은? (1-hop)

# run_query("""
#     MATCH ...
#     RETURN ...
# """)

In [ ]:
# [연습 2-2] B공급사가 공급하는 재료를 사용하는 모든 공정은? (2-hop)

# run_query("""
#     MATCH ...
#     RETURN ...
# """)

In [ ]:
# [연습 2-3] 크랙 불량의 원인 장비를 사용하는 공정에서 필요한 재료의 공급사는? (3-hop)

# run_query("""
#     MATCH ...
#     RETURN ...
# """)

---
## 6. 정리 + 다음 Part 미리보기

### 오늘 배운 것

| 항목 | 내용 |
|------|------|
| Neo4j 연결 | Python `neo4j` 드라이버로 연결 |
| CREATE | 노드와 관계 생성 |
| MATCH + RETURN | 패턴 매칭으로 데이터 조회 |
| WHERE | 조건부 필터링 |
| SET / DELETE | 수정과 삭제 |
| Multi-hop 쿼리 | 1-hop, 2-hop, 3-hop 경로 탐색 |
| GraphRAG의 필요성 | 벡터 검색의 한계와 그래프의 강점 |

### 핵심 인사이트

> **"1-hop이면 벡터로 충분하다. Multi-hop이 필요하면 GraphRAG를 고려하라."**

### 다음 Part 2 미리보기: 수작업 KG 구축

Part 2에서는 다음을 배웁니다:

1. **온톨로지 설계** - 엔티티 타입과 관계 타입을 체계적으로 정의
2. **수동 엔티티 추출** - 텍스트에서 직접 엔티티와 관계를 추출
3. **Meta-Dictionary** - 관계 키워드 사전 만들기
4. **Neo4j 적재** - 추출 결과를 그래프로 변환

Part 1에서 7개 노드를 만들었다면, Part 2에서는 뉴스 기사 10개에서 **수십 개의 노드**를 추출합니다!

In [ ]:
# 세션 정리: 드라이버 종료
# 실습이 모두 끝난 후 실행하세요.
# (다음 실습을 이어서 할 경우에는 실행하지 마세요)

# driver.close()
# print("[OK] Neo4j 드라이버 종료 완료")

print("Part 1 실습을 마칩니다.")
print("수고하셨습니다! Part 2에서 만나요!")